Boyd spell checker is only on individual words, i. e. token dataframe was build by replicating the suggestion list of respective types. 


The token dataframe is therefore in a different order than it is for Language Tool, where context sensitive information is used and the frame can be used to refer back to the original text.

***
1. **Get suggestion list**
    - **Load both dataframes**
        - Get suggestion list from spell-checker
        - Get token+offset information dataframe (from lt)
    - **Merge dataframes**
        - Merge Boyd suggestions (type frame, case sensitive) with dataframe that has all the text information (Language Tool token frame + text information) on 'original', 'corrected'
        - Does not work with token dataframe (...); Key is not unique (...)
    - **Clean up dataframes**    
        - By that token dataframe is reconstructed that has all the text information + suggestions by Boyd
2. **Build chunks for n-gram reranking**
    - Get full original texts
    - Prep
    - Loop via function
***
- Note: It is important to align indices of dataframes to be able to loop over it and reconstruct later on
    - full texts + data_error_token_offsets (OTHERINDEX_20211208) align on index that refers to index for each token within full texts

In [1]:
import pandas as pd
import sys

sys.path.insert(0, '../../Lisa')
import litkey_2

# Do not truncate columns of DataFrame
pd.set_option('display.max_rows', None)
# Do not truncate rows of DataFrame
pd.set_option('display.max_rows', None)

import helpers_reranking as helpers

## 1 - Get suggestion list
- Load both dataframes
    - Get suggestion list from spell-checker
    - Get token+offset information dataframe (from lt)
- Merge dataframes
- Clean up dataframes

### 1.1 Load both dataframes

In [2]:
# Load Boyd suggestions via evaluation dataframe (CS, type)
boyd_suggs = pd.read_pickle(r'../Experiment_Boyd/data_error_types_boyd_evaluation_cs.pkl')

In [3]:
#boyd_suggs.tail(20)
boyd_suggs.shape[0] #9484

9484

In [4]:
# Get token + text information via language tool dataframe (CS, token)
# WITH OFFSET INFORMATION
data_error_token_offsets = pd.read_pickle('../Experiment_language_tool/pickles_evaluation/data_error_token_language_tool_whole_texts_evaluation_cs_ALL_COLUMNS_OFFSETS_OTHERINDEX_20211208.pkl')

In [5]:
# Save index of this dataframe to column
data_error_token_offsets['idx_df_orig'] = data_error_token_offsets.index

In [6]:
data_error_token_offsets.tail(10)
data_error_token_offsets.shape[0] #24601

24601

### 1.2 Merge dataframes

- Boyd suggestions, type level (boyd_suggs)
- token df with all text information (from LT) (data_error_token_offsets)

- on original corrected
    - filename doesn't matter, because Boyd has been given types only and token df has been reconstructed from that
- KEEP INDEX_TEXT

In [8]:
# Merge
boyd_text_inf = data_error_token_offsets.merge(boyd_suggs, how='left', on=['original','corrected'], suffixes=('','_y'))

In [9]:
# Check shape
boyd_text_inf.shape[0]

24601

### 1.3 Clean up dataframes
- Keep text information
- And evaluation information (debug purposes)

In [10]:
boyd_text_inf.head(2)

,index,original,corrected,filename,num_texts,length,start_position,end_position,ct,suggestions,...,idx0,idx_df_orig,filename_y,freq_ori_y,freq_cor_y,freq_tup_y,suggestions_y,in_sugg_y,sugg_idx_y,idx0_y
0,10,belt,bellt,01-005-2-III-Eis.csv,0,4,46,50,10,"[Belt, Welt, bald, hält, Geld, Feld, Held, Zel...",...,False,0,01-005-2-III-Eis.csv,91,138,91,"[ballt, bellt, beult, bellst, hebelt, beeilt, ...",True,1.0,False
1,17,kukt,kuckt,01-005-2-III-Eis.csv,0,4,80,84,17,"[rückt, zückt, bückt, guckt, kickt, kackt, gut...",...,False,1,01-005-2-III-Eis.csv,73,152,73,"[Guckt, kuckt, kickt, guckt, Guckst, guckst, k...",True,1.0,False


In [11]:
# Get rid of evaluation information (in_sugg, sugg_idx, idx0) x & y
# Keep text information (and evlaluation information)

unn = ['in_sugg', 'sugg_idx', 'idx0', 'suggestions', 'rule_id', 'freq_ori_y', 'freq_cor_y', 'freq_tup_y', 'filename_y', 'in_sugg_y', 'sugg_idx_y', 'idx0_y']
boyd_text_inf.drop(columns=unn, inplace=True)

In [12]:
# Suggestion column is now Boyds!
boyd_text_inf.rename(columns={'suggestions_y':'suggestions'}, inplace=True)

In [13]:
boyd_text_inf.tail(10)

,index,original,corrected,filename,num_texts,length,start_position,end_position,ct,freq_ori,freq_cor,freq_tup,idx_df_orig,suggestions
24591,210710,nieht,nicht,10-693-4-IV-Weg.csv,1921,5,721,726,145,2,135,2,24591,"[niet, nicht, niest, näht, Nicht, sieht, zieht..."
24592,210711,vessen,fassen,10-693-4-IV-Weg.csv,1921,6,727,733,146,1,2,1,24592,"[Wessen, wessen, fesseln, fressen, vermessen, ..."
24593,210715,entlich,endlich,10-693-4-IV-Weg.csv,1921,7,744,751,150,10,20,10,24593,"[endlich, Endlich, sämtlich, endlisch, ernstli..."
24594,210730,Tor,Tür,10-693-4-IV-Weg.csv,1921,3,820,823,165,1,15,1,24594,"[Tor, Tors, Thor, Torf, Tore, Toren, Tour, Ten..."
24595,210732,umarte,umarmte,10-693-4-IV-Weg.csv,1921,6,828,834,167,2,16,2,24595,"[umarmte, Umarmte, umarmten, rumorte, unartige..."
24596,210743,nieht,nicht,10-693-4-IV-Weg.csv,1921,5,886,891,178,2,135,2,24596,"[niet, nicht, niest, näht, Nicht, sieht, zieht..."
24597,210749,ass,aß,10-693-4-IV-Weg.csv,1921,3,912,915,184,3,7,3,24597,"[Ass, aß, nass, pass, lass, bass, fass, dass, ..."
24598,210752,mutzte,nutzte,10-693-4-IV-Weg.csv,1921,6,923,929,187,1,1,1,24598,"[nutzte, motzte, benutzte, muckste, nützte, nu..."
24599,210754,momen,Moment,10-693-4-IV-Weg.csv,1921,5,934,939,189,1,4,1,24599,"[Moment, Nomen, Moments, mopsen, Momenten, Mom..."
24600,210760,Lutscha,Lutscher,10-693-4-IV-Weg.csv,1921,7,962,969,195,8,56,8,24600,"[Lutscher, lutsche, Lutschen, lutschten, Lutsc..."


In [14]:
# To merge with for postprocessing
# Naming convention: preface + "preprocessed_reranking"

#boyd_text_inf.to_pickle('data_error_types_boyd_evaluation_cs_preprocessed_reranking.pkl')

In [15]:
# !!! Cannot maintain index through merge, therefore set new index at this point !!!

# Rename index column
boyd_text_inf.rename(columns={'index':'index_text'}, inplace=True)
# Set index to be able to better work with in loop
boyd_text_inf.set_index('index_text', inplace=True)

In [16]:
data_error_token_offsets.shape[0] # 24601
boyd_text_inf.shape[0] # 24601

24601

***

## 2 - Build chunks for n-gram reranking
- Get full original texts
- Prep
- Loop

### 2.1 Get full original texts

In [17]:
# Get original texts resp. preprocessed, i. e. raw error token (i. e. Input for Language Tool)
# WITH OFFSETS (see postpr_evaluation_whole_texts)

#ref_list = pd.read_pickle('../Experiment_language_tool/pickles/data_raw_token_whole_texts_offsets.pkl')
ref_list = pd.read_pickle('../Experiment_language_tool/pickles/data_raw_token_whole_texts_offsets_OTHERINDEX_20211208.pkl')

In [18]:
ref_list.head(10)

,index,original,corrected,filename,num_texts,length,start_position,end_position,ct
0,0,Lea,Lea,01-005-2-III-Eis.csv,0,3,0,3,0
1,1,und,und,01-005-2-III-Eis.csv,0,3,4,7,1
2,2,Lars,Lars,01-005-2-III-Eis.csv,0,4,8,12,2
3,3,und,und,01-005-2-III-Eis.csv,0,3,13,16,3
4,4,Dodo,Dodo,01-005-2-III-Eis.csv,0,4,17,21,4
5,5,gehen,gehen,01-005-2-III-Eis.csv,0,5,22,27,5
6,6,Eis,Eis,01-005-2-III-Eis.csv,0,3,28,31,6
7,7,kaufen,kaufen,01-005-2-III-Eis.csv,0,6,32,38,7
8,8,.,.,01-005-2-III-Eis.csv,0,1,39,40,8
9,9,Dodo,Dodo,01-005-2-III-Eis.csv,0,4,41,45,9


### 2.2 Prep

In [19]:
# Copy index to column
ref_list['idx_per_text'] = ref_list.index

ref_list  = ref_list.rename(columns={'index':'index_text'})
ref_list_ = ref_list.set_index('index_text')
ref_list_.head(10)
#ref_list_.tail(20)

# Index resets with every new file
# Index-column is ongoing

,original,corrected,filename,num_texts,length,start_position,end_position,ct,idx_per_text
index_text,,,,,,,,,
0,Lea,Lea,01-005-2-III-Eis.csv,0,3,0,3,0,0
1,und,und,01-005-2-III-Eis.csv,0,3,4,7,1,1
2,Lars,Lars,01-005-2-III-Eis.csv,0,4,8,12,2,2
3,und,und,01-005-2-III-Eis.csv,0,3,13,16,3,3
4,Dodo,Dodo,01-005-2-III-Eis.csv,0,4,17,21,4,4
5,gehen,gehen,01-005-2-III-Eis.csv,0,5,22,27,5,5
6,Eis,Eis,01-005-2-III-Eis.csv,0,3,28,31,6,6
7,kaufen,kaufen,01-005-2-III-Eis.csv,0,6,32,38,7,7
8,.,.,01-005-2-III-Eis.csv,0,1,39,40,8,8


In [20]:
ref_list_.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            210761, 210762, 210763, 210764, 210765, 210766, 210767, 210768,
            210769, 210770],
           dtype='int64', name='index_text', length=210771)

***

### 2.3 Loop _via function_
- File is written in Input_chunk

In [21]:
speller = 'boyd'

In [22]:
helpers.create_chunk_file(suggl_by=speller, reference_df=ref_list_, speller_df=boyd_text_inf)